In [1]:
import requests
import json

In [7]:
x_hubspot_csrf_hubspotapi = input("Csrf_token: ")

In [8]:
cookie_value = input("Cookie: ")

In [9]:
userid = input("Userid: ")

In [10]:
import requests
import json

contacts_api_url = "https://api.hubapi.com/crm/v3/objects/contacts"
ACCESS_TOKEN = ''
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}',
    'Content-Type': 'application/json'
}

def get_contacts():
    all_contacts = []
    after = None
    
    while True:
        params = {
            'properties': 'email,additionalEmails'
        }
        if after:
            params['after'] = after
            
        response = requests.get(contacts_api_url, headers=headers, params=params)
        data = response.json()
        
        contacts = data.get('results', [])
        all_contacts.extend(contacts)
        
        paging = data.get('paging')
        if paging and 'next' in paging:
            after = paging['next']['after']
        else:
            break
    
    return all_contacts

In [12]:
contacts = get_contacts()
contact_id = 132916927299
emails = []
email_to_delete = "johnny_allen1103061267+1@fuliss.net"

In [14]:
for contact in contacts:
    additional_emails = contact['properties'].get('additionalEmails')
    if additional_emails:
        contact_id = contact['id']
        
        contact_emails = additional_emails.split(';')
        for email in contact_emails:
            if email.strip():
                emails.append({
                    'contact_id': contact_id,
                    'email': email.strip()
                })

In [15]:
url = f"https://app.hubspot.com/api/graphql/crm?portalId={userid}&clienttimeout=60000&hs_static_app=crm-records-ui&hs_static_app_version=1.76446"

headers = {
    "x-hubspot-csrf-hubspotapi": x_hubspot_csrf_hubspotapi,
    "Content-Type": "application/json",
    "Cookie": cookie_value
}

payload = {
    "operationName": "PropertiesCardDeleteEmail",
    "variables": {
        "deleteEmailInput": {
            "objectId": contact_id,
            "secondaryEmailAddress": email_to_delete
        },
        "propertyNames": [
            "hs_additional_emails"
        ]
    },
    "query": """mutation PropertiesCardDeleteEmail($deleteEmailInput: DeleteContactSecondaryEmailAddressInput!, $propertyNames: [String!]!) {
  updateResponse: deleteContactSecondaryEmailAddress(input: $deleteEmailInput) {
    userErrors
    updatedObject {
      id
      properties(names: $propertyNames) {
        id
        ...PropertiesCardPropertyValueFragment
        __typename
      }
      __typename
    }
    __typename
  }
}

fragment PropertiesCardPropertyValueFragment on PropertyValue {
  id
  definition {
    ...PropertiesCardPropertyDefinitionFragment
    __typename
  }
  value
  name
  source
  permission {
    ...PropertiesCardFieldLevelPermissionFragment
    __typename
  }
  __typename
}

fragment PropertiesCardPropertyDefinitionFragment on PropertyDefinition {
  name
  objectTypeId
  label
  deleted
  hidden
  hubspotDefined
  dataSensitivity
  __typename
}

fragment PropertiesCardFieldLevelPermissionFragment on FieldLevelPermission {
  accessLevel
  __typename
}"""
}

def delete_secondary_email(contact_id, email_to_delete):
    payload["variables"]["deleteEmailInput"]["objectId"] = contact_id
    payload["variables"]["deleteEmailInput"]["secondaryEmailAddress"] = email_to_delete
    
    try:
        response = requests.post(url, headers=headers, json=payload)
        
        if response.status_code == 200:
            result = response.json()
            print(f"Email '{email_to_delete}' deleted from {contact_id}")
            return result
        else:
            print(f"Error {response.status_code}: {response.text}")
            return None
            
    except Exception as e:
        print(f"Error {str(e)}")
        return None

In [16]:
for email_data in emails:
    contact_id = email_data['contact_id']
    email_to_delete = email_data['email']
    
    print(f"Eliminando email '{email_to_delete}' del contacto {contact_id}")
    delete_secondary_email(contact_id, email_to_delete)